In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from pathlib import Path

# ---------- CONFIGURE THIS SECTION ----------
PG_HOST = "localhost"
PG_PORT = 5432
PG_DB   = "ciasom"
PG_USER = "postgres"
PG_PASS = "****"
# -------------------------------------------

ROOT = Path(__file__).resolve().parents[1]
excel_path = ROOT / "data" / "processed" / "CiASOM.xlsx"

print(f"Loading Excel from: {excel_path}")

# 1. Load the 'All' sheet
df = pd.read_excel(excel_path, sheet_name="All")

# 2. Rename columns to SQL-friendly snake_case
rename_map = {
    "Project": "project",
    "Sample CiASOM ID": "sample_ciasom_id",
    "Sample MOSAiC ID": "sample_mosaic_id",
    "PS Operation Device/Lable on Bottle": "ps_operation_label",
    "Leg": "leg",
    "Week": "week",
    "Location": "location",
    "Snow Thickness (cm)": "snow_thickness_cm",
    "Latitude": "latitude",
    "Longitude": "longitude",
    "Media_Type": "media_type",
    "Date": "sample_date",
    "Time": "sample_time",
    "Week_MOSAiC": "week_mosaic",
    "Air_Temp": "air_temp",
    "In Core Temp (°C)": "in_core_temp_c",
    "Sample Depth (cm)": "sample_depth_cm",
    "Depth_1": "depth_1",
    "Depth_2": "depth_2",
    "Mean_Depth": "mean_depth",
    "Salinity (psu)": "salinity_psu",
    "d18O": "d18o",
    "dD": "dd",
    "d_excess": "d_excess",
    "Comments": "comments",
}

df = df.rename(columns=rename_map)

# 3. Create a combined timestamp
df["sample_datetime"] = pd.to_datetime(
    df["sample_date"].astype(str) + " " + df["sample_time"].astype(str),
    errors="coerce"
)

# 4. Basic type cleaning
for col in ["leg", "week", "week_mosaic"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

numeric_cols = [
    "snow_thickness_cm", "latitude", "longitude", "air_temp",
    "in_core_temp_c", "sample_depth_cm", "depth_1", "depth_2",
    "mean_depth", "salinity_psu", "d18o", "dd", "d_excess"
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 5. Connect to Postgres
conn = psycopg2.connect(
    host=PG_HOST,
    port=PG_PORT,
    dbname=PG_DB,
    user=PG_USER,
    password=PG_PASS,
)
conn.autocommit = True
cur = conn.cursor()

# 6. Drop + recreate table
cur.execute("DROP TABLE IF EXISTS ciasom_samples;")

create_table_sql = """
CREATE TABLE ciasom_samples (
    project             TEXT,
    sample_ciasom_id    TEXT,
    sample_mosaic_id    TEXT,
    ps_operation_label  TEXT,
    leg                 INTEGER,
    week                INTEGER,
    location            TEXT,
    snow_thickness_cm   DOUBLE PRECISION,
    latitude            DOUBLE PRECISION,
    longitude           DOUBLE PRECISION,
    media_type          TEXT,
    sample_date         DATE,
    sample_time         TIME,
    week_mosaic         INTEGER,
    air_temp            DOUBLE PRECISION,
    in_core_temp_c      DOUBLE PRECISION,
    sample_depth_cm     DOUBLE PRECISION,
    depth_1             DOUBLE PRECISION,
    depth_2             DOUBLE PRECISION,
    mean_depth          DOUBLE PRECISION,
    salinity_psu        DOUBLE PRECISION,
    d18o                DOUBLE PRECISION,
    dd                  DOUBLE PRECISION,
    d_excess            DOUBLE PRECISION,
    comments            TEXT,
    sample_datetime     TIMESTAMP
);
"""
cur.execute(create_table_sql)

# 7. Insert data
cols = list(df.columns)
records = df.where(pd.notnull(df), None).to_records(index=False)
values = [tuple(r) for r in records]

insert_sql = f"""
INSERT INTO ciasom_samples ({", ".join(cols)})
VALUES %s;
"""

print(f"Inserting {len(values)} rows into ciasom_samples...")
execute_values(cur, insert_sql, values)

cur.close()
conn.close()

print("Done. Table ciasom_samples created in PostgreSQL.")